In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

import codecs

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
# Convert .csv from Win-1251 to utf-8
f = codecs.open('../data/dump_19_22.csv', 'r', 'cp1251')
df = pd.read_csv(f, sep=';', encoding='utf-8')

df.columns = pd.Series(df.columns).apply(lambda x: x.lower().replace(' ', '_'))

df.head()

,номер_животного,ушная_бирка_животного,пол,номер_лактации,результат_отела,легкость_отела,дата_рождения,дней_в_сухостое_предыдущей_лактации,дней_стельности_при_событии,номер_группы_животного,предыдущий_номер_группы_животного,событие,дни_доения_при_событии,дата_события,примечание_события
0,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,РАСЧКОП,336,28.03.2019,КАН
1,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,КОГЛАВАК
2,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,ЛЕПТО
3,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023
4,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПРОДАНА,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8


In [5]:
df_moves = df[df.событие == 'ПЕРЕВОД'].copy()

df_moves.head()

,номер_животного,ушная_бирка_животного,пол,номер_лактации,результат_отела,легкость_отела,дата_рождения,дней_в_сухостое_предыдущей_лактации,дней_стельности_при_событии,номер_группы_животного,предыдущий_номер_группы_животного,событие,дни_доения_при_событии,дата_события,примечание_события
3,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023
7,321,896820302,F,6,FA,3,11.09.2009,62,291,18,18,ПЕРЕВОД,216,18.03.2019,F006T003
10,321,896820302,F,6,FA,3,11.09.2009,62,291,18,18,ПЕРЕВОД,231,02.04.2019,F004T012
17,321,896820302,F,6,FA,3,11.09.2009,62,291,18,18,ПЕРЕВОД,252,23.04.2019,F004T012
34,641,532029621,F,6,FA,1,01.01.2010,63,0,23,23,ПЕРЕВОД,138,20.03.2019,F011T042


In [6]:
df_moves.примечание_события.str.len().value_counts()

8    114094
Name: примечание_события, dtype: int64

In [7]:
df_moves.примечание_события.str[0].value_counts()

F    114094
Name: примечание_события, dtype: int64

In [8]:
df_moves.примечание_события.str[4].value_counts()

T    114094
Name: примечание_события, dtype: int64

In [9]:
def from_transitions_parser(trans_log):
    return int(trans_log[1:4])

def to_transition_parser(trans_log):
    return int(trans_log[5:])

In [10]:
test_log = df_moves.iloc[0]['примечание_события']
print(f'Log to parse:\t', test_log)

print(from_transitions_parser(test_log), to_transition_parser(test_log))

Log to parse:	 F001T023
1 23


In [11]:
df_moves[['номер_группы_животного', 'предыдущий_номер_группы_животного']] = \
    df_moves.примечание_события.apply([from_transitions_parser, to_transition_parser])

In [12]:
df_moves.head()

,номер_животного,ушная_бирка_животного,пол,номер_лактации,результат_отела,легкость_отела,дата_рождения,дней_в_сухостое_предыдущей_лактации,дней_стельности_при_событии,номер_группы_животного,предыдущий_номер_группы_животного,событие,дни_доения_при_событии,дата_события,примечание_события
3,291,530073354,F,6,MA,1,30.07.2009,68,0,1,23,ПЕРЕВОД,448,08.04.2019,F001T023
7,321,896820302,F,6,FA,3,11.09.2009,62,291,6,3,ПЕРЕВОД,216,18.03.2019,F006T003
10,321,896820302,F,6,FA,3,11.09.2009,62,291,4,12,ПЕРЕВОД,231,02.04.2019,F004T012
17,321,896820302,F,6,FA,3,11.09.2009,62,291,4,12,ПЕРЕВОД,252,23.04.2019,F004T012
34,641,532029621,F,6,FA,1,01.01.2010,63,0,11,42,ПЕРЕВОД,138,20.03.2019,F011T042


In [18]:
df_moves['группа_изменена'] = 1

df_moves.группа_изменена.value_counts()

1    114094
Name: группа_изменена, dtype: int64

In [16]:
select_df = ['номер_животного', 'номер_лактации', 'дней_в_сухостое_предыдущей_лактации'
                  ,  'дней_стельности_при_событии', 'событие' 
                  , 'дата_события','примечание_события', 'группа_изменена']
merge_on = ['номер_животного', 'номер_лактации', 'дней_в_сухостое_предыдущей_лактации'
        ,'дней_стельности_при_событии', 'событие', 'дата_события','примечание_события']

In [19]:
df_updated = pd.merge(
    df, df_moves[select_df],
    on=merge_on
    , how='left'
)

df_updated.head()

,номер_животного,ушная_бирка_животного,пол,номер_лактации,результат_отела,легкость_отела,дата_рождения,дней_в_сухостое_предыдущей_лактации,дней_стельности_при_событии,номер_группы_животного,предыдущий_номер_группы_животного,событие,дни_доения_при_событии,дата_события,примечание_события,группа_изменена
0,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,РАСЧКОП,336,28.03.2019,КАН,NaN
1,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,КОГЛАВАК,NaN
2,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,ЛЕПТО,NaN
3,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023,1.0
4,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПРОДАНА,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8,NaN


In [20]:
df.shape[0] - df_moves.shape[0] - df_updated.группа_изменена.isna().sum()

0

In [21]:
df_updated.группа_изменена = df_updated.группа_изменена.fillna(0)

df_updated.isna().sum().sum()

260950